In [ ]:
"""
DEVELOPING SGD LEVY ALGORITHM - starting 03/04/2022 notebook 2
"""

In [1]:
%load_ext autoreload
%autoreload 2
import ObjectiveFunction as of
from SGDLevy import SGDLevy
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import helper_funcs as hf
from scipy.stats import levy_stable, uniform
from math import sqrt

In [2]:
"""
alpha = 2 - beta (tail index)
look at flat landscape, check the levy walk

TO DO:
implement ensemble MSD -> eMSD
use polar coordinates - levy sample for magnitude, uniform sample for direction angle
send FIMA paper to Pulin

LEARNED: 
levy distribution is special case of levy-alpha-stable distribution 
with stability alpha = 0.5 and skew beta = 1 (+required scale c and location mu parameters)
    - scale parameter --> smaller makes step size dist decay faster and peak at a lower size
"""

'\nalpha = 2 - beta (tail index)\nlook at flat landscape, check the levy walk\n\nTO DO:\nimplement ensemble MSD -> eMSD\nuse polar coordinates - levy sample for magnitude, uniform sample for direction angle\nsend FIMA paper to Pulin\n\nLEARNED: \nlevy distribution is special case of levy-alpha-stable distribution \nwith stability alpha = 0.5 and skew beta = 1 (+required scale c and location mu parameters)\n    - scale parameter --> smaller makes step size dist decay faster and peak at a lower size\n'

In [3]:
# LEVY WALK SIMULATION
def levy_flight(x0, alpha, beta, n, upperbound, scale=1):
    rs = levy_stable.rvs(alpha=alpha, beta=beta, size=n, loc=0, scale=scale)
    rs = rs[rs < upperbound*scale]
    thetas = uniform.rvs(loc=0, scale=2*np.pi, size=len(rs))
    xs = list(np.cumsum(rs*np.cos(thetas)))
    ys = list(np.cumsum(rs*np.sin(thetas)))
    xs.insert(0, x0[0])
    ys.insert(0, x0[1])

    return [xs, ys], rs

path, rs = levy_flight(np.array([0, 0]), alpha=0.5, beta=1,
                       n=3000, upperbound=100, scale=1)

In [6]:
figx1, taus, tmsds = hf.tmsd_analysis(path, max_tau_ratio=0.99)
alphas = hf.derivative(np.log(taus), np.log(tmsds))
figx2 = hf.plot(taus, tmsds, '𝜏', 'tMSD')
figx3 = hf.plot(taus, alphas, '𝜏', 'α')
figx4 = go.Figure()
figx4.add_trace(
    go.Histogram(x=rs)
)
figx4.update_layout(
    autosize=False,
    width=1600,
    height=800,
    title=len(rs))
hf.figures_to_html([figx1, figx2, figx3, figx4], 'levy_walk_tmsd_analysis.html')

C:\Users\karls\AppData\Local\Temp/ipykernel_5392/2314159402.py:2: RuntimeWarning:

divide by zero encountered in log



In [2]:
m = of.AlpineN1(start=[2.5,2.5])
alpline_sgdlevy = of.OptimisationProblem(
    m,
    SGDLevy(m.parameters(), lr=0.01, momentum=0.90, noise_pc=8e-2, alpha=2),
    n_epochs = 1000
)

losses, params, preds = alpline_sgdlevy.run()
fig = alpline_sgdlevy.visualise((-5, 6), (-6, 6), 0.1, render="contour")
fig.show()

In [6]:
float(levy_stable.rvs(alpha=0.5, beta=1, size=1, loc=0, scale=0.18))

0.2029575248894085

In [ ]:
"""
Check if changing alpha gives you super-sub diffusive
Implement periodic boundary conditions

read the papers he sent
- escaping local minima (essential)
- getting convergence from broken ergodicity
- simulated tempering
- throwing ropes over mountains
make summary of each

send summary of equations 

idea:
- superdiffusive in areas where you haven't spent a lot of time, 
- sub-diffusive in areas where you have spent a lot of time

store local minima - strong nuclear force term - only significant when close in z
- repels when below, and attracts when above
"""

In [ ]:
"""
25/04/2022
Read adam paper
Read problems adam has
Read improvements to adam
Read problems with these improvements
Try find one that might need adaptive diffusion behaviour
"""

In [ ]:
"""
01/05/2022
Implement LEVY FLIGHT attempt
- levy flight, but normal dist of angles centered around angle of current gradient (NALF)
- ADAM, but use angular distribution so far to inform directions of current step
- run levy flight simulations, and check the relationship between diffusion coefficients and alpha
"""

In [5]:
def cart2polar(x):
    x = np.array(x)
    return np.arctan2(x[1], x[0])

cart2polar(torch.Tensor([1,1])) * 180/np.pi

45.00000125223908

In [65]:
from torch.optim.optimizer import Optimizer, required
import torch
from typing import List, Optional
from scipy.stats import uniform, norm

class SGD_NALF(Optimizer):

    def __init__(self,
                 params,
                 alpha: float = required,
                 lr: float = required,
                 angle_spread: float = required,
                 levy_spread: float = required,
                 momentum: float = 0
    ):

        if lr is not required and lr < 0.0:
            raise ValueError(f"Invalid learning rate: {lr}")
        if momentum < 0.0:
            raise ValueError(f"Invalid momentum value: {momentum}")
        
        defaults = dict(lr=lr,
                        momentum=momentum,
                        alpha=alpha,
                        angle_spread=angle_spread,
                        levy_spread=levy_spread)
        super().__init__(params, defaults)

    @torch.no_grad()
    def step(self, closure=None):

        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()
        
        # Get the current parameter state

        for group in self.param_groups:
            for p in group['params']:
                grad = p.grad

                if 'momentum_buffer' not in self.state:
                    self.state['momentum_buffer'] = grad.detach().clone()
                else:
                    self.state['momentum_buffer'].mul_(group['momentum']).add_(grad, alpha=1)
                grad = self.state['momentum_buffer']
                p.add_(grad, alpha=-group['lr'])

                # NALF
                alpha = group['alpha']
                levy_r = hf.levyrandom(alpha, beta=1, mu=0.0, sigma=group['levy_spread']) * torch.norm(grad)
                cur_dir = cart2polar(grad)
                theta = float(norm.rvs(loc=cur_dir, scale=group['angle_spread']))
                dir = np.array([np.cos(theta), np.sin(theta)])
                levy_noise = levy_r * torch.Tensor(dir)
                p.add_(levy_noise, alpha=-group['lr'])


        return loss

In [66]:
m = of.AlpineN1(start=[2.5, 2], bounds=10)
alpline_NALF = of.OptimisationProblem(
    m,
    SGD_NALF(m.parameters(), alpha=2, lr=0.01, momentum=0.90, angle_spread=0.15, levy_spread=0.1),
    n_epochs = 1000
)

losses, params, preds = alpline_NALF.run()
fig = alpline_NALF.visualise((-10, 10), (-10, 10), 0.1, render="contour")
fig.show()

In [72]:
m = of.Ackley(start=[1.0, 1.2], bounds=10)
ackley_NALF = of.OptimisationProblem(
    m,
    SGD_NALF(m.parameters(), alpha=2, lr=0.01, momentum=0.90, angle_spread=0.15, levy_spread=0.01),
    n_epochs = 1000
)

losses, params, preds = ackley_NALF.run()
fig = ackley_NALF.visualise((-10, 10), (-10, 10), 0.1, render="contour")
fig.show()

In [50]:
levysamples =  np.array([np.mean([hf.levyrandom(1.0, 1.0, 1, 0.05) for _ in range(1000)]) for __ in range(100)])
fig = go.Figure()
fig.add_trace(
    go.Histogram(x=levysamples)
)
fig.show()

In [55]:
np.mean([hf.levyrandom(0.5, 1.0, 1.0, 0.05) for _ in range(1000)])

247.4092256350946

In [33]:
def levy_flight(x0, alpha, beta, n, upperbound, scale=1):
    rs = hf.levyrandom(alpha=alpha, beta=beta, mu=0.0, sigma=scale, shape=(n,1))
    #rs = levy_stable.rvs(alpha=alpha, beta=beta, size=n, loc=0, scale=scale)
    #rs = rs[rs < upperbound*scale]
    thetas = uniform.rvs(loc=0, scale=2*np.pi, size=len(rs))
    xs = list(np.cumsum(rs*np.cos(thetas)))
    ys = list(np.cumsum(rs*np.sin(thetas)))
    xs.insert(0, x0[0])
    ys.insert(0, x0[1])

    return [xs, ys], rs

path, rs = levy_flight(np.array([0, 0]), alpha=0.5, beta=1,
                       n=3000, upperbound=100, scale=1)

In [75]:
paths = []
rss = []
for _ in range(0, 100):
    path, rs = levy_flight(np.array([0, 0]), alpha=1.0, beta=1,
                       n=1000, upperbound=100, scale=1)
    paths.append(path)
    rss.append(rs)
    if _ % 10 == 0: print(_)

c:\Users\karls\OneDrive\Documents\GitHub\honours\helper_funcs.py:389: RuntimeWarning:

invalid value encountered in power



0
10
20
30
40
50
60
70
80
90


In [76]:
fig1, ts, emsds = hf.emsd_analysis(paths, tau=20)

c:\Users\karls\OneDrive\Documents\GitHub\honours\helper_funcs.py:186: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

C:\Users\karls\AppData\Local\Temp/ipykernel_14376/1401239702.py:2: RuntimeWarning:

divide by zero encountered in log



TypeError: derivative() got an unexpected keyword argument 'window'

In [78]:
alphas = hf.derivative(np.log(ts), np.log(emsds), h=20)
fig2 = hf.plot(ts, emsds, 't', 'eMSD')
fig3 = hf.plot(ts, alphas, 't', 'α')

C:\Users\karls\AppData\Local\Temp/ipykernel_14376/552126314.py:1: RuntimeWarning:

divide by zero encountered in log



In [80]:
hf.figures_to_html([fig1,fig2, fig3], 'levy_walk_simulation_emsd.html')

In [40]:
rs = hf.levyrandom(alpha=0.5, beta=1.0, mu=0.0, sigma=1.0, shape=(100000,1))

In [41]:
sum(np.isnan(rs))

array([0])